# Exercise

- Use the previous knowledge to extract information from the European Environment Agency (EEA). 
- Adopt the monitoring co2 emissions [dataset](http://www.eea.europa.eu/data-and-maps/data/co2-cars-emission-11) (**Monitoring of CO2 emissions from passenger cars - Data 2015 - Final data**) 

# 1 - Hipótese
Será que a potencia dos motores influenciam na emissão de CO2 ?

In [30]:
#%%timeit
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn import linear_model
from pprint import pprint
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.charts import Bar
from bokeh.models.widgets import Tabs, Panel

In [2]:
pd_cars = pd.read_csv("versao3.csv", low_memory = False, error_bad_lines = False, warn_bad_lines = False, encoding = "utf-8")
fuelfilter = ['E85', 'DIESEL-ELECTRIC', 'PETROL-ELECTRIC', 'ELECTRIC', 'NG-BIOMETHANE', 'LPG', 'PETROL', 'DIESEL']

In [37]:
def Filter():
    df_filter = []
    for index, row in pd_cars.iterrows():
        try:
            if (str(row["ep (KW)"]).isnumeric()) and (str(row["e (g/km)"]).isnumeric()) and fuelfilter.index(str(row["Ft"])):
                df_filter.append(row)
        except:
            continue

    df_filter = pd.DataFrame(df_filter)        
    len(df_filter)
    df_filter.to_csv("versao3.csv",  encoding = "utf-8")

# Return array 2D
def CountValues(index, valMin):
    listReturn = []
    for i, row in pd_cars[index].value_counts().sort_values().iteritems():
        if(row > valMin):
            listReturn.append([i, row, 0, 0])
    np_listReturn = np.array(listReturn)   
    return np_listReturn

# 2 - Fabricantes
np_mk é um array 2d que fica o nome das fabricantes e a quantidade de carros que elas fizeram. 
Esse array tem 8 fabricantes e 178.518 carros. Todas as fabricantes tem mais que 10.000 carros produzidos.

In [38]:
np_mk = CountValues("Mk", 10000)

data = {
    'Fabricantes': np_mk[:,0],
    'Quantidade de carros': np_mk[:,1]
}

bar = Bar(data, values='Quantidade de carros', label='Fabricantes', agg="mean",
          title="Fabricantes de carros", legend=False, plot_width=900)

output_notebook()
show(bar)

Loading BokehJS ...

# 3 - Correlação
Foi aplicado a função *pearsonr* para calcular o coeficiente de correlação da potecia sobre emissão.<br>
O coeficiente foi elevado ao quadrado para extrair o coeficiente de determinação, este mede o percentual de variação da emissão que é explicado pela variação da potencia.

In [39]:
for row in np_mk:    
    x = pd_cars[pd_cars["Mk"] == row[0]]["ep (KW)"]
    y = pd_cars[pd_cars["Mk"] == row[0]]["e (g/km)"]
    row[2] = pearsonr(x, y)[0]
    row[3] = (pearsonr(x, y)[0]**2)*100
    
first = np_mk[np_mk[:, 2].argsort()][:2]
last = np_mk[np_mk[:, 2].argsort()][-2:]
np_mk_job = np.concatenate((first, last), axis=0)

# 4 - Regressão Linear e apresentação dos resultados
Foi aplicado a função *LinearRegression* para saber a função que relaciona as variáveis e com isso poder plotar a reta de aproximação. <br>
4 gráficos foram plotados, eles representam as 2 motadores com os menores R² e as 2 com os maiores R². Nas labels das tabs foi informado o R²

In [40]:
listTabs = []
for row in np_mk_job:
    x_values = pd_cars[pd_cars["Mk"] == row[0]][["ep (KW)"]]
    y_values = pd_cars[pd_cars["Mk"] == row[0]][["e (g/km)"]]
    #train model on data
    bmi_life_model = linear_model.LinearRegression()
    bmi_life_model.fit(x_values, y_values)
    predict = bmi_life_model.predict(x_values).ravel()

    x = pd_cars[pd_cars["Mk"] == row[0]]["ep (KW)"]
    y = pd_cars[pd_cars["Mk"] == row[0]]["e (g/km)"]

    p = figure(plot_width=800, plot_height=600, title=row[0], x_axis_label='ep (KW)', y_axis_label='e (g/km)')

    p.line(x, predict, line_width=2)
    p.circle(x, y, fill_color="white", size=5)
    label = str(row[0]) + "  |  R² = " + str(row[3])[0:5]
    listTabs.append(Panel(child=p, title=label))

layout = Tabs(tabs=listTabs)

output_notebook()
show(layout)

Loading BokehJS ...

In [41]:
pot = []
for row in np_mk_job:
    listAux = []
    for i, rowPot in pd_cars[pd_cars["Mk"] == row[0]]["ep (KW)"].value_counts()[0:6].iteritems():
        listAux.append([i, rowPot])
    pot.append(listAux)

pot = np.array(pot)

# 5 - Potencia dos motores
Com base nos resultados apresentados eu as 6 potencias de motores mais utilizados por cada montadora e plotei no gráfico para saber se tinha alguma relação com os dados anteriores. <br>

In [42]:
from bokeh.charts import Line

data = dict(VOLKSWAGEN = np.sort(pot[0][:,0]),
            SKODA = np.sort(pot[1][:,0]),
            AUDI = np.sort(pot[2][:,0]),
            BMW = np.sort(pot[3][:,0])
            )
df = pd.DataFrame(data)

line = Line(df, y=['VOLKSWAGEN', 'SKODA', 'AUDI', 'BMW'],
             dash=['VOLKSWAGEN', 'SKODA', 'AUDI', 'BMW'],
             title="Interpreters", ylabel='Potencia', legend=True, plot_width=800)

show(line)

# 6 - Conclusão
Foi notado que as montadoras com motores mais potentes tem uma correlação menor do que as que possuem um motor menos potente. <br>
O resultado foi o esperado, mas ainda é preciso mais analises para poder comprovar essa relação.